<a href="https://colab.research.google.com/github/duchaba/Data-Augmentation-with-Python/blob/main/data_augmentation_with_python_chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌻 Welcome to Chapter 7, Audio Data Augmentation

---

I am glad to see you using this Python Notebook. 🐕

The Python Notebook is an integral part of the book. You can add new “code cells” to extend the functions, add your data, and explore new possibilities, such as downloading additional real-world datasets from the Kaggle website and coding the **Fun challenges**. Furthermore, the book has **Fun facts**, in-depth discussion about augmentation techniques, and Pluto, an imaginary Siberian Huskey coding companion. Together they will guide you every steps of the way.

Pluto encourages you to copy or save a copy of this Python Notebook to your local space and add the “text cells” to keep your notes. In other words, read the book and copy the relevant concept to this Python Notebook’s text-cells. Thus, you can have the explanation, note, original code, your code, and any crazy future ideas in one place.  


💗 I hope you enjoy reading the book and hacking code as much as I enjoy writing it.


## 🌟 Amazon Book

---

- The book is available on the Amazon Book website:
  - https://www.amazon.com/dp/1803246456

  - Author: Duc Haba
  - Published: 2023
  - Page count: 390+


- The original Python Notebook is on:
  - https://github.com/PacktPublishing/Data-Augmentation-with-Python/blob/main/Chapter_7/data_augmentation_with_python_chapter_7.ipynb

- 🚀 Click on the blue "Open in Colab" button at the top of this page to begin hacking.


# 😀 Excerpt from Chapter 7, Audio Data Augmentation

---

> In case you haven’t bought the book. Here is an teaser from the first page of Chapter 7.

---

Similar to image and text augmentation, the objective of audio data augmentation is to extend the dataset to gain a higher accuracy forecast or prediction in a generative AI system. Audio augmentation is cost-effective and is a viable option when acquiring additional audio files is expensive or time-consuming.  

Writing about audio augmentation methods poses unique challenges. The first is that audio is not visual like images or text. If the format is audiobooks, web pages, or mobile apps, then we play the sound, but the medium is paper. Thus, we must transform the audio signal into a visual representation. The Waveform graph, also known as the time series graph, is a standard method for representing an audio signal. You can listen to the audio in the accompanying Python Notebook.

In this chapter, you will learn how to write Python code to read an audio file and draw a Waveform graph from scratch. Pluto has provided a preview here so that we can discuss the components of the Waveform graph. The function is as follows:

> ...skip...image...

In Figure 7.1, Pluto drew the positive amplitude in blue and the negative amplitude in yellow in the Waveform graph. This makes the chart easier to read and prettier. The amplitude is the value of the Y-axis. It measures the vibration or compression and decompression of the air molecules. The higher the amplitude, the greater the air displacement. In other words, the zero amplitude value is silent, and the greater the absolute distance from zero, the louder the sound.

The frequency, also known as the sampling rate, is the value of the X-axis. The sampling rate measures how many times you recorded the amplitude value in a second. The unit for sound frequency or the sampling rate is hertz (Hz). For example, a sampling rate of 1,000 Hz or 1 kilohertz (kHz) means you record a thousand amplitude values in 1 second. In other words, you register an amplitude value for every millisecond. Thus, the higher the frequency, the more accurate the sound, and therefore, a larger sound file size. This is because there is a higher recorded amplitude value. 1 kHz is equal to 1,000 Hz.

Pitch is the same as frequency but from a human point of view. It refers to the loudness of the sound and is measured in decibels (dB). Thus, high pitch means high frequency.

dB is the unit for the degree of loudness. A rocket sound is about 165 dB, busy traffic noise is about 85 dB, human speech is about 65 dB, rainfall is about 45 dB, and zero dB means silence.

The standard sampling rate for MP3 and other audio formats is 22.05 kHz. The frequency of high-quality sound, also known as Compact Disk (CD) sound, is 44.1 kHz.

When storing an audio file on a computer, bit depth is the accuracy of the amplitude value. 16 bits has 65,536 levels of detail, while 24 bits has 16,777,216 levels of information. The higher the bit depth, the closer the digital recording is to the analog sound and the larger the audio file size.  

The bit rate is similar to the sampling rate, where the bit rate measures the number of bits per second. In audio processing, the playback function uses the bit rate, while the record function uses the sampling rate.

Mono sound has one channel (1-channel), while stereo sound has two channels (2-channel). Stereo sound has one channel for the right ear and another channel for the left ear.

> ...skip...skip...

This chapter will cover audio augmentation using Waveform transformation, and in particular, the following topics:   

- Standard audio augmentation techniques

- Filters

- Audio augmentation libraries

- Real-world audio datasets  

- Reinforcing your learning through Python Notebook

Let’s begin by discussing common audio augmentation methods.

---

🌴 *end of excerpt from the book*

# Load Notebook


- This Notebook original link is:
  - https://github.com/PacktPublishing/Data-Augmentation-with-Python/blob/main/data_augmentation_with_python_chapter_7.ipynb

# GitHub Clone

In [1]:
# git version should be 2.17.1 or higher
!git --version

git version 2.34.1


In [2]:
url = 'https://github.com/PacktPublishing/Data-Augmentation-with-Python'
!git clone {url}

Cloning into 'Data-Augmentation-with-Python'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (448/448), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 470 (delta 244), reused 355 (delta 172), pack-reused 22
Receiving objects: 100% (470/470), 139.54 MiB | 4.25 MiB/s, done.
Resolving deltas: 100% (244/244), done.
Updating files: 100% (58/58), done.


## Fetch file from URL (Optional)

- Uncommend the below 2 code cells if you want to use URL and not Git Clone

In [ ]:
# import requests
# #
# def fetch_file(url, dst):
#   downloaded_obj = requests.get(url)
#   with open(dst, "wb") as file:
#     file.write(downloaded_obj.content)
#   return

In [ ]:
# url = ''
# dst = 'pluto_chapter_1.py'
# fetch_file(url,dst)

# Run Pluto

- Instantiate up Pluto, aka. "Pluto, wake up!"

In [3]:
# %% CARRY-OVER code install

!pip install opendatasets
!pip install pyspellchecker

# The tested version:
# opendatasets==0.1.12
# pyspellchecker==0.7.1
#
# if error with newer version, then install:
# !pip install opendatasets==0.1.12
# !pip install pyspellchecker==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.2 MB/s eta 0:00:00


In [4]:
#load and run the pluto chapter 1 Python code.
pluto_file = 'Data-Augmentation-with-Python/pluto/pluto_chapter_2.py'
%run {pluto_file}

---------------------------- : ----------------------------
            Hello from class : <class '__main__.PacktDataAug'> Class: PacktDataAug
                   Code name : Pluto
                   Author is : Duc Haba
---------------------------- : ----------------------------


## Verify Pluto

In [5]:
pluto.say_sys_info()

---------------------------- : ----------------------------
                 System time : 2023/10/17 22:13
                    Platform : linux
     Pluto Version (Chapter) : 2.0
             Python (3.7.10) : actual: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
            PyTorch (1.11.0) : actual: 2.0.1+cu118
              Pandas (1.3.5) : actual: 1.5.3
                 PIL (9.0.0) : actual: 9.4.0
          Matplotlib (3.2.2) : actual: 3.7.1
                   CPU count : 2
                   CPU speed : 2.20 GHz
               CPU max speed : 0.00 GHz
---------------------------- : ----------------------------


## (Optional) Export to .py

In [ ]:
pluto_chapter_7 = 'Data-Augmentation-with-Python/pluto/pluto_chapter_7.py'
!cp {pluto_file} {pluto_chapter_7}

# ✋ Set up Kaggle username and app Key

- Install the following libraries, and import it on the Notebook.
- Follow by initialize Kaggle username, key and fetch methods.

- STOP: Update your Kaggle access username or key first.

In [6]:
# %%CARRY-OVER code

# -------------------- : --------------------
# READ ME
# Chapter 2 begin:
# Install the following libraries, and import it on the Notebook.
# Follow by initialize Kaggle username, key and fetch methods.
# STOP: Update your Kaggle access username or key first.
# -------------------- : --------------------

!pip install opendatasets --upgrade
import opendatasets
print("\nrequired version 0.1.22 or higher: ", opendatasets.__version__)

!pip install pyspellchecker
import spellchecker
print("\nRequired version 0.7+", spellchecker.__version__)

# STOP: Update your Kaggle access username or key first.
pluto.remember_kaggle_access_keys("YOUR-USERNAME", "YOUR-KEY")
pluto._write_kaggle_credit()
import kaggle

@add_method(PacktDataAug)
def fetch_kaggle_comp_data(self,cname):
  #self._write_kaggle_credit()  # need to run only once.
  path = pathlib.Path(cname)
  kaggle.api.competition_download_cli(str(path))
  zipfile.ZipFile(f'{path}.zip').extractall(path)
  return

@add_method(PacktDataAug)
def fetch_kaggle_dataset(self,url,dest="kaggle"):
  #self._write_kaggle_credit()    # need to run only once.
  opendatasets.download(url,data_dir=dest)
  return
# -------------------- : --------------------



required version 0.1.22 or higher:  0.1.22

Required version 0.7+ 0.7.2


# Fetch Kaggle Data

## Musical Emotions Classification

In [ ]:
%%time
url = 'https://www.kaggle.com/datasets/kingofarmy/musical-emotions-classification'
pluto.fetch_kaggle_dataset(url)

In [ ]:
f = 'kaggle/musical-emotions-classification/Train.csv'
pluto.df_music_data = pluto.fetch_df(f)
pluto.df_music_data.head(3)

In [ ]:
# %%writefile -a {pluto_chapter_7}

pluto.version = 7.0
# augment full path
@add_method(PacktDataAug)
def _append_music_full_path(self,x):

  """
  Append the full path of the music file to the dataframe.

  Args:
    x: A string containing the name of the music file.

  Returns:
    A string containing the full path of the music file.
  """

  y = re.findall('([a-zA-Z ]*)\d*.*', x)[0]
  return (f'kaggle/musical-emotions-classification/Audio_Files/Audio_Files/Train/{y}/{x}')
#
@add_method(PacktDataAug)
def fetch_music_full_path(self, df):

  """
  Append the full path of the music file to the dataframe.

  Args:
    df: A pandas dataframe containing the music file names.

  Returns:
    A pandas dataframe with the full path of the music files appended.
  """

  df['fname'] = df.ImageID.apply(self._append_music_full_path)
  return df

In [ ]:
pluto.fetch_music_full_path(pluto.df_music_data)
pluto.df_music_data.head(3)

## Human Speech

In [ ]:
%%time
url = 'https://www.kaggle.com/datasets/ejlok1/cremad'
pluto.fetch_kaggle_dataset(url)

In [ ]:
#
# change method name to make_dir_dframe
f = 'kaggle/cremad/AudioWAV'
pluto.df_voice_data = pluto.make_dir_dataframe(f)
pluto.df_voice_data.head(3)

## Urban sound

In [ ]:
%%time
url = 'https://www.kaggle.com/datasets/rupakroy/urban-sound-8k'
pluto.fetch_kaggle_dataset(url)

In [ ]:
#
# change method name to make_dir_dframe
f = 'kaggle/urban-sound-8k/UrbanSound8K/UrbanSound8K/audio'
pluto.df_sound_data = pluto.make_dir_dataframe(f)
pluto.df_sound_data.head(3)

# View and Listen

In [ ]:
# to-do: there is a deprecated function with version 0.10.0 (Feb. 2023)
!pip install librosa==0.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.1
    Uninstalling librosa-0.10.1:
      Successfully uninstalled librosa-0.10.1


In [ ]:
# %%writefile -a {pluto_chapter_7}

import IPython
import IPython.display
import librosa
import librosa.display
import matplotlib
import pandas
import numpy
import re

In [ ]:
# %%writefile -a {pluto_chapter_7}

# prompt: write inline documentation with default value and type for the following python function: _draw_area_with_neg()
@add_method(PacktDataAug)
def _draw_area_with_neg(self,ndata,pic,tcolor='#007bff',bcolor='#ffc107',alpha=0.75,istart=0,izero=0):

  """
  Draw a positive/negative area with a line chart.

  Args:
    ndata: (numpy array) A numpy array containing the data to be plotted.
    pic: (figure) A matplotlib.pyplot.Figure object to draw the plot on.
    tcolor: (str) The color of the positive area. Default is '#007bff'
    bcolor: (str) The color of the negative area. Default is '#ffc107'
    alpha: (float) The transparency of the areas. Default is 0.75
    istart: (int) The starting index of the data to be plotted. Default is zero.
    izero: (int) The zero value of the data. Default is zero.

  Returns:
    None.
  """

  nlen = len(ndata)
  i = numpy.arange(istart, istart+nlen)
  xzero = numpy.zeros(nlen)
  xzero += izero
  # plot line
  pic.plot(i,ndata, linewidth=0.0)  # invisible line for shading
  pic.plot(i,xzero,color='gray',linewidth=0.5) # base line

  # fill top/positive section
  pic.fill_between(
    i, xzero, ndata, where=(ndata >= xzero),
    interpolate=True, color=tcolor, alpha=alpha,
    label="Positive"
  )
  # fill bottom/negative section
  pic.fill_between(
    i, xzero, ndata, where=(ndata < xzero),
    interpolate=True, color=bcolor, alpha=alpha,
    label="Negative"
  )
  return
#
# prompt: write inline documentation with default value and type for the following python function: _fetch_audio_data()
@add_method(PacktDataAug)
def _fetch_audio_data(self,lname):

  """
  Fetch audio data from a file name.

  Args:
    lname: (str) The file name of the audio file.

  Returns:
    (tuple) A tuple containing the audio data, the sampling rate, and the file name.
  """

  data_amp, sam_rate = librosa.load(lname, mono=True)
  fname = re.findall('[^\/]+$', lname)
  return data_amp, sam_rate, fname[0]
#
# prompt: write inline documentation with default value and type for the following python function: _draw_audio()
@add_method(PacktDataAug)
def _draw_audio(self,data_amp, sam_rate, fname):

  """
  Draw audio waveform.

  Args:
    data_amp (numpy array): , 1 dimensional.
    sam_rate (integer): sampling rate.
    fname: string): name of the output file.

  Return:
    None
  """

  # define constant
  zlen = 100
  nrow = 2
  ncol = 1
  w = 11
  h = 6
  title = ['', 'Zoom In: From Midpoint to 100+']
  ylabel = ['Waveform Amplitude']
  #
  # define graph
  canvas, pic = matplotlib.pyplot.subplots(nrow, ncol, figsize=(w, h))
  pics = pic.flatten()
  # draw original
  dlen = len(data_amp)
  self._draw_area_with_neg(data_amp,pic[0])
  pics[0].set_title(f'{title[0]} {fname}', fontsize=18.)
  pics[0].set_xlabel(f'Second: Total {dlen/sam_rate:.2f} sec., Sampling Rate: {sam_rate/1000:.2f} kHz', fontsize=16.)
  pics[0].set_ylabel(ylabel[0])
  #
  # overide tick with time
  loc = numpy.array(pics[0].get_xticks())
  b = loc / sam_rate
  b = numpy.round(b,1)
  pics[0].set_xticklabels(b)
  pics[0].grid()
  #
  # draw zoom
  mid = int(len(data_amp) / 2)
  end = mid + zlen
  self._draw_area_with_neg(data_amp[mid:end],pic[1])
  pics[1].set_title(title[1],fontsize=18.)
  pics[1].set_xlabel(f'Time Series Index: Midpoint at {mid/sam_rate:.2f} sec. ({mid} ts)', fontsize=16.)
  pics[1].set_ylabel(ylabel[0])
  pics[1].grid()
  #
  # display and save image
  canvas.tight_layout()
  self._drop_image(canvas)
  canvas.show()
  #
  return
#
# prompt: write inline documentation with default value and type for the following python function: draw_audio()
@add_method(PacktDataAug)
def draw_audio(self,df,id_name=None):

  """
  Draw audio waveform.

  Args:
    df: (pandas DataFrame): with column 'fname' that contains the file name of
      each audio file.
    id_name (string): file name of the audio file to draw. If None, a random
      file name will be drawn.

  Returns:
    None.
  """
  # random take a file name
  if (id_name is None):
    samp = df.sample(1)
    samp.reset_index(drop=True, inplace=True)
    in_file = samp.fname[0]
  else:
    in_file = id_name
  #
  data_amp, sam_rate, fname = self._fetch_audio_data(in_file)
  self._draw_audio(data_amp, sam_rate, 'Original: ' + fname)
  # display audio
  display(IPython.display.Audio(data_amp,rate=sam_rate))
  return
#

In [ ]:
pluto.draw_audio(pluto.df_music_data)

In [ ]:
pluto.draw_audio(pluto.df_voice_data)

In [ ]:
pluto.draw_audio(pluto.df_sound_data, id_name=p)

# Audio Control D-scale

- pitches D, E, F♯, G, A, B, and C♯

In [ ]:
#
f = 'Data-Augmentation-with-Python/pluto_data/control-d-major.mp3'
data_amp, sam_rate, fname = pluto._fetch_audio_data(f)
pluto.audio_control_dmajor = [data_amp, sam_rate, fname, f]
pluto._draw_audio(data_amp, sam_rate, 'Original: ' + fname)
# display audio
display(IPython.display.Audio(data_amp, rate=sam_rate))

# Time shift

In [8]:
# %CARRY-OVER

!pip install audiomentations
#
# tested with version 0.29.0
# !pip install audiomentations==0.29.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 1.9 MB/s eta 0:00:00


In [10]:
# %%writefile -a {pluto_chapter_7}

import audiomentations
#
@add_method(PacktDataAug)
def _fetch_1_sample(self, df, dsize=1):

  """
  Fetch a sample from the given DataFrame.

  Parameters:
  - df (pd.DataFrame): The input DataFrame from which a sample is to be fetched.
  - dsize (int, optional): The size of the sample to fetch. Default is 1.

  Returns:
  - str: The 'fname' value from the sampled row.

  Note:
  This function assumes that the DataFrame 'df' has a column named 'fname'.
  """
  p = df.sample(dsize)
  p.reset_index(drop=True, inplace=True)
  return p.fname[0]
#
@add_method(PacktDataAug)
def _audio_transform(self, df, xtransform, title=''):

  """
  Apply audio transformation and display the result.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    xtransform: (audiomentations.core.BaseTransformer), audio transformation to apply.
    title: (string), title of the plot. Default value is empty string.

  Returns: None.
  """
  if (type(df) is list):
    data_amp, sam_rate, fname, lname = self.audio_control_dmajor
  else:
    lname = self._fetch_1_sample(df)
    data_amp, sam_rate, fname = self._fetch_audio_data(lname)
  #
  xaug = xtransform(data_amp, sample_rate=sam_rate)
  # augmented
  self._draw_audio(xaug, sam_rate, title + ' Augmented: ' + fname)
  display(IPython.display.Audio(xaug, rate=sam_rate))
  # original
  self._draw_audio(data_amp, sam_rate, 'Original: ' + fname)
  display(IPython.display.Audio(data_amp, rate=sam_rate))
  return

In [12]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_time_shift(self, df,
  min_fraction=-0.2,
  max_fraction=0.8,
  rollover=True,
  title='Time Shift',
  id_name=None):

  """
  Play audio samples with time shift augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_fraction: (float), minimum fraction of the audio to shift. Default value is -0.2.
    max_fraction: (float), maximum fraction of the audio to shift. Default value is 0.8.
    rollover: (bool), If True, the audio is shifted so that the beginning of the audio is played
      at the end and vice versa. Default value is True.
    title: (string), title of the plot. Default value is empty string.
    id_name: (string), the column name that contains the unique identifier of the audio file. If `None`,
      the 'fname' column is used. Default value is None.

  Returns: None.
  """

  xtransform = audiomentations.Shift(
    min_fraction = min_fraction,
    max_fraction = max_fraction,
    rollover = rollover,
    p=1.0
  )
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_time_shift(pluto.df_music_data, min_fraction=0.5)

In [ ]:
pluto.play_aug_time_shift(pluto.df_voice_data, min_fraction=0.5)

In [ ]:
pluto.play_aug_time_shift(pluto.df_sound_data, min_fraction=0.5)

In [ ]:
pluto.play_aug_time_shift(pluto.audio_control_dmajor, min_fraction=0.2)

# Time stretch

In [15]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_time_stretch(self, df, min_rate=0.2,max_rate=6.8,leave_length_unchanged=True,title='Time Stretch'):

  """
  Play audio samples with time stretch augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_rate: (float), minimum rate to use for time stretching. Default value is 0.2.
    max_rate: (float), maximum rate to use for time stretching. Default value is 6.8.
    leave_length_unchanged: (bool), If True, the length of the audio is preserved by changing the
      sample rate. Default value is True.
    title: (string), title of the plot. Default value is "Time Stretch".

  Returns: None.
  """

  xtransform = audiomentations.TimeStretch(
    min_rate = min_rate,
    max_rate = max_rate,
    leave_length_unchanged = leave_length_unchanged,
    p=1.0
  )
  self._audio_transform(df, xtransform, title=title)
  return
  # librosa.effects.time_stretch under the hood

In [ ]:
pluto.play_aug_time_stretch(pluto.audio_control_dmajor, max_rate=5.4)

In [ ]:
pluto.play_aug_time_stretch(pluto.df_music_data, max_rate=3.0)

In [ ]:
pluto.play_aug_time_stretch(pluto.df_voice_data, max_rate=3.5)

In [ ]:
pluto.play_aug_time_stretch(pluto.df_sound_data, max_rate=2.5)

In [ ]:
# pluto.fname_id

# Pitch scaling

In [17]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_pitch_scaling(self, df, min_semitones=-6.0,max_semitones=6.0,title='Pitch Scaling'):

  """
  Play audio samples with pitch scaling augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_semitones: (float), minimum number of semitones to pitch shift. Default value is -6.0.
    max_semitones: (float), maximum number of semitones to pitch shift. Default value is 6.0.
    title: (string), title of the plot. Default value is "Pitch Scaling".

  Returns: None.
  """

  xtransform = audiomentations.PitchShift(
    min_semitones = min_semitones,
    max_semitones = max_semitones,
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_pitch_scaling(pluto.audio_control_dmajor,min_semitones=-6.0,max_semitones=6.0)

In [ ]:
# pluto.fname_id

In [ ]:
pluto.play_aug_pitch_scaling(pluto.df_music_data,min_semitones=-11.0, max_semitones=-9.0)

In [ ]:
pluto.play_aug_pitch_scaling(pluto.df_voice_data,min_semitones=-6.0, max_semitones=6.0)

In [ ]:
pluto.play_aug_pitch_scaling(pluto.df_sound_data,min_semitones=4.0, max_semitones=11.0)

# Noise injection, add Gaussian noise

In [19]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_noise_injection(self, df, min_amplitude=0.002,max_amplitude=0.2,title='Gaussian noise injection'):

  """
  Play audio samples with noise injection (Gaussian noise) augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_amplitude: (float), minimum amplitude of the noise to add. Default value is 0.002.
    max_amplitude: (float), maximum amplitude of the noise to add. Default value is 0.2.
    title: (string), title of the plot. Default value is "Gaussian noise injection".

  Returns: None.
  """

  xtransform = audiomentations.AddGaussianNoise(
    min_amplitude = min_amplitude,
    max_amplitude = max_amplitude,
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_noise_injection(pluto.audio_control_dmajor, min_amplitude=0.02,max_amplitude=0.05)

In [ ]:
# pluto.fname_id

In [ ]:
pluto.play_aug_noise_injection(pluto.df_music_data, max_amplitude=0.01)

In [ ]:
pluto.play_aug_noise_injection(pluto.df_voice_data, max_amplitude=0.01)

In [ ]:
pluto.play_aug_noise_injection(pluto.df_sound_data, max_amplitude=0.022)

# Polarity Inversion

In [21]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_polar_inverse(self, df, title='Polarity inversion'):

  """
  Play audio samples with polarity inversion augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    title: (string), title of the plot. Default value is "Polarity inversion".

  Returns: None.
  """

  xtransform = audiomentations.PolarityInversion(
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_polar_inverse(pluto.audio_control_dmajor)

In [ ]:
pluto.play_aug_polar_inverse(pluto.df_voice_data)

# Low pass Filter

In [24]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_low_pass_filter(self, df,
  min_cutoff_freq=150, max_cutoff_freq=7500,
  min_rolloff=12, max_rolloff=24,
  title='Low pass filter'):

  """
  Play audio samples with low pass filter augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_cutoff_freq: (float), minimum cutoff frequency in Hz. Default value is 150.
    max_cutoff_freq: (float), maximum cutoff frequency in Hz. Default value is 7500.
    min_rolloff: (float), minimum rolloff in dB/octave. Default value is 12.
    max_rolloff: (float), maximum rolloff in dB/octave. Default value is 24.
    title: (string), title of the plot. Default value is "Low pass filter".

  Returns: None.
  """

  xtransform = audiomentations.LowPassFilter(
    min_cutoff_freq = min_cutoff_freq,
    max_cutoff_freq = max_cutoff_freq,
    min_rolloff = min_rolloff,
    max_rolloff = max_rolloff,
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_low_pass_filter(pluto.audio_control_dmajor)

In [ ]:
pluto.play_aug_low_pass_filter(pluto.df_music_data)

In [ ]:
pluto.play_aug_low_pass_filter(pluto.df_voice_data)

In [ ]:
pluto.play_aug_low_pass_filter(pluto.df_sound_data)

#High pass filter

In [25]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_high_pass_filter(self, df,
  min_cutoff_freq=20, max_cutoff_freq=2400,
  min_rolloff=12, max_rolloff=24,
  title='High pass filter'):

  """
  Play audio samples with high pass filter augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_cutoff_freq: (float), minimum cutoff frequency in Hz. Default value is 20.
    max_cutoff_freq: (float), maximum cutoff frequency in Hz. Default value is 2400.
    min_rolloff: (float), minimum rolloff in dB/octave. Default value is 12.
    max_rolloff: (float), maximum rolloff in dB/octave. Default value is 24.
    title: (string), title of the plot. Default value is "High pass filter".

  Returns: None.
  """

  xtransform = audiomentations.HighPassFilter(
    min_cutoff_freq = min_cutoff_freq,
    max_cutoff_freq = max_cutoff_freq,
    min_rolloff = min_rolloff,
    max_rolloff = max_rolloff,
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_high_pass_filter(pluto.audio_control_dmajor)

In [ ]:
pluto.play_aug_high_pass_filter(pluto.df_music_data)

In [ ]:
pluto.play_aug_high_pass_filter(pluto.df_voice_data)

In [ ]:
pluto.play_aug_high_pass_filter(pluto.df_sound_data)

# Band pass filter

In [27]:
# %%writefile -a {pluto_chapter_7}

@add_method(PacktDataAug)
def play_aug_band_pass_filter(self, df,
  min_center_freq=200, max_center_freq=4000,
  min_bandwidth_fraction=0.5,max_bandwidth_fraction=1.99,
  min_rolloff=12, max_rolloff=24,
  title='Band pass filter'):

  """
  Play audio samples with band pass filter augmentation.

  Args:
    df: (pandas DataFrame), with column 'fname' that contains the file name of each audio file.
    min_center_freq: (float), minimum center frequency in Hz. Default value is 200.
    max_center_freq: (float), maximum center frequency in Hz. Default value is 4000.
    min_bandwidth_fraction: (float), minimum bandwidth fraction. Default value is 0.5.
    max_bandwidth_fraction: (float), maximum bandwidth fraction. Default value is 1.99.
    min_rolloff: (float), minimum rolloff in dB/octave. Default value is 12.
    max_rolloff: (float), maximum rolloff in dB/octave. Default value is 24.
    title: (string), title of the plot. Default value is "Band pass filter".

  Returns: None.
  """

  xtransform = audiomentations.BandPassFilter(
    min_center_freq = min_center_freq,
    max_center_freq = max_center_freq,
    min_bandwidth_fraction = min_bandwidth_fraction,
    max_bandwidth_fraction = max_bandwidth_fraction,
    min_rolloff = min_rolloff,
    max_rolloff = max_rolloff,
    p=1.0)
  self._audio_transform(df, xtransform, title=title)
  return

In [ ]:
pluto.play_aug_band_pass_filter(pluto.audio_control_dmajor)

In [ ]:
pluto.play_aug_band_pass_filter(pluto.df_music_data)

In [ ]:
pluto.play_aug_band_pass_filter(pluto.df_voice_data)

In [ ]:
pluto.play_aug_band_pass_filter(pluto.df_sound_data)

# Sneak peek (Optional)

In [ ]:
pluto.text_marry_lamb = 'Mary had a little lamb, whose fleece was white as snow. And everywhere that Mary went, the lamb was sure to go.'

In [ ]:
x = numpy.array([ord(c) for c in pluto.text_marry_lamb])

In [ ]:
k = pandas.DataFrame(x, columns=['tdata'])
k.head()

In [ ]:
pluto.fname_id=0

In [ ]:
canvas, pic = matplotlib.pyplot.subplots(1, 1, figsize=(11, 4))
# pics = pic.flatten()
pluto._draw_area_with_neg(x,pic,izero=64)
pic.set_title('Text time-series graph: Marry had a little lamb...', fontsize=18.)
pic.set_xlabel('Character, letters are in blue, punctuations are in yellow', fontsize=16.)
pic.set_ylabel('ASCII value')
pic.grid()
#
# display and save image
canvas.tight_layout()
pluto._drop_image(canvas)
canvas.show()

In [ ]:
print('End of chapter 7')

# Push up all changes (Optional)

- username: duchaba

- password: [use the token]

In [ ]:
# import os
# f = 'Data-Augmentation-with-Python'
# os.chdir(f)
# !git add -A
# !git config --global user.email "duc.haba@gmail.com"
# !git config --global user.name "duchaba"
# !git commit -m "end of session"
# # do the git push in the xterm console
# #!git push

In [ ]:
# import google.colab
# import datetime
# @add_method(PacktDataAug)
# def zip_download(self,dname='/content/Data-Augmentation-with-Python/pluto_img'):
#   d = datetime.datetime.now()
#   d = d.strftime('%Y%m%d_')
#   fname = f'img{d}{self.fname_id}.zip'
#   !zip {fname} {dname}/*
#   google.colab.files.download(fname)
#   return
# #

In [ ]:
# pluto.zip_download()

# Summary

Every chaper will begin with same base class "PacktDataAug".

✋ FAIR WARNING:

- The coding uses long and complete function path name.

- Pluto wrote the code for easy to understand and not for compactness, fast execution, nor cleaverness.

- Use Xterm to debug cloud server



In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm
# %xterm